# Comprehensive SyftBox Permissions Tutorial
## Part 3: Inheritance and Hierarchy

Welcome to Part 3! This is where SyftBox permissions become truly powerful. You'll learn how permissions flow through directory trees and the sophisticated nearest-node algorithm that makes it all work.

### What You'll Learn
- The nearest-node algorithm in detail
- Multi-level inheritance scenarios
- Permission accumulation vs. overrides
- Complex directory tree navigation
- Debugging inheritance issues

### Prerequisites
- Completed Part 1: Fundamentals
- Completed Part 2: Patterns and Matching
- Understanding of directory tree structures

In [ ]:
import syft_perm as sp
from pathlib import Path
import tempfile
import shutil
import yaml

# Setup workspace
tutorial_dir = Path(tempfile.mkdtemp(prefix="inheritance_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="inheritance_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path, title=None):
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        display_title = title or f"syft.pub.yaml in {path.name}"
        print(f"\n=== {display_title} ===")
        print(yaml_file.read_text())
        print("=" * 50)
    else:
        print(f"No syft.pub.yaml in {path}")

def trace_inheritance(file_path, user, permission):
    """Trace how a permission is inherited up the directory tree"""
    print(f"\n=== Tracing {permission} permission for {user} ===")
    print(f"File: {file_path.relative_to(tutorial_dir)}")
    
    syft_file = sp.open(file_path)
    has_perm, reasons = syft_file._check_permission_with_reasons(user, permission)
    
    print(f"Result: {has_perm}")
    print("Reasoning:")
    for reason in reasons:
        print(f"  • {reason}")
    
    return has_perm

## Chapter 1: The Nearest-Node Algorithm Deep Dive

The **nearest-node algorithm** is the heart of SyftBox permissions. When checking a file's permissions, SyftBox walks up the directory tree looking for the **nearest parent directory** that has matching rules.

### Key Principle: **Single Node Selection, Not Accumulation**

In [ ]:
# Create a multi-level directory structure to demonstrate the algorithm
# Structure:
# tutorial_dir/ (root)
#   ├── level1/
#   │   ├── level2/
#   │   │   ├── level3/
#   │   │   │   └── deep_file.txt
#   │   │   └── mid_file.txt
#   │   └── shallow_file.txt
#   └── root_file.txt

structure = {
    "root_file.txt": tutorial_dir / "root_file.txt",
    "level1/shallow_file.txt": tutorial_dir / "level1" / "shallow_file.txt",
    "level1/level2/mid_file.txt": tutorial_dir / "level1" / "level2" / "mid_file.txt", 
    "level1/level2/level3/deep_file.txt": tutorial_dir / "level1" / "level2" / "level3" / "deep_file.txt"
}

for rel_path, full_path in structure.items():
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(f"Content of {rel_path}")

print("Created directory structure:")
print(tutorial_dir.name + "/")
print("├── root_file.txt")
print("└── level1/")
print("    ├── shallow_file.txt")
print("    └── level2/")
print("        ├── mid_file.txt")
print("        └── level3/")
print("            └── deep_file.txt")

In [ ]:
# Scenario 1: Only root has permissions
print("=== Scenario 1: Only Root Has Permissions ===")

# Create permissions only at the root level
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "**"
  access:
    read:
    - alice@example.com
""")

show_yaml(tutorial_dir, "Root permissions")

# Test inheritance for each file
for rel_path, full_path in structure.items():
    has_access = trace_inheritance(full_path, "alice@example.com", "read")
    print(f"  → {rel_path}: {'✓' if has_access else '✗'}")

In [ ]:
# Scenario 2: Add permissions at level2 - demonstrates nearest-node
print("\n=== Scenario 2: Add Permissions at Level2 ===")

# Add permissions at level2 (different user)
level2_yaml = tutorial_dir / "level1" / "level2" / "syft.pub.yaml"
level2_yaml.write_text("""rules:
- pattern: "**"
  access:
    write:
    - bob@example.com
""")

show_yaml(tutorial_dir, "Root permissions (unchanged)")
show_yaml(tutorial_dir / "level1" / "level2", "Level2 permissions (new)")

print("\nTesting nearest-node algorithm:")
for rel_path, full_path in structure.items():
    print(f"\n{rel_path}:")
    alice_read = trace_inheritance(full_path, "alice@example.com", "read")
    bob_write = trace_inheritance(full_path, "bob@example.com", "write")
    print(f"  Alice (read): {'✓' if alice_read else '✗'}")
    print(f"  Bob (write): {'✓' if bob_write else '✗'}")

In [ ]:
# Analyze the results
print("\n=== Nearest-Node Algorithm Analysis ===")
print()
print("Files and their nearest nodes:")
print("• root_file.txt → Root (tutorial_dir/)")
print("• shallow_file.txt → Root (tutorial_dir/)")
print("• mid_file.txt → Level2 (level1/level2/)")
print("• deep_file.txt → Level2 (level1/level2/)")
print()
print("Key insight: Files inherit from the NEAREST parent with matching rules.")
print("Level2 rules OVERRIDE root rules for files in level2/ and below.")
print("This is NOT accumulation - it's replacement!")

## Chapter 2: Pattern Matching in Inheritance

The nearest-node algorithm doesn't just find the nearest directory with rules - it finds the nearest directory with **matching** rules.

In [ ]:
# Create a new scenario with specific patterns
cleanup_and_reset()

# Create files with different extensions
files = {
    "docs/guide.md": "Documentation",
    "docs/api/reference.md": "API docs", 
    "src/main.py": "Python code",
    "src/utils/helper.py": "Utility code",
    "config/settings.json": "Configuration"
}

for rel_path, content in files.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

print("Created files:")
for rel_path in files.keys():
    print(f"  {rel_path}")

In [ ]:
# Set up permissions with specific patterns at different levels

# Root: Python files only
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "**/*.py"
  access:
    read:
    - python_devs@example.com
""")

# Docs: Markdown files only
docs_yaml = tutorial_dir / "docs" / "syft.pub.yaml"
docs_yaml.write_text("""rules:
- pattern: "**/*.md"
  access:
    read:
    - doc_writers@example.com
""")

show_yaml(tutorial_dir, "Root: Python files")
show_yaml(tutorial_dir / "docs", "Docs: Markdown files")

print("\n=== Pattern-Specific Inheritance Test ===")

In [ ]:
# Test each file
test_cases = [
    ("docs/guide.md", "doc_writers@example.com", True, "Matches *.md in docs/"),
    ("docs/api/reference.md", "doc_writers@example.com", True, "Matches *.md in docs/"),
    ("src/main.py", "python_devs@example.com", True, "Matches *.py in root"),
    ("src/utils/helper.py", "python_devs@example.com", True, "Matches *.py in root"),
    ("config/settings.json", "python_devs@example.com", False, "No matching pattern anywhere")
]

for rel_path, user, expected, explanation in test_cases:
    full_path = tutorial_dir / rel_path
    syft_file = sp.open(full_path)
    has_access = syft_file.has_read_access(user)
    
    status = "✓" if has_access == expected else "✗ WRONG"
    print(f"{status} {rel_path} → {user}: {has_access}")
    print(f"    {explanation}")
    
    # Show reasoning for failing cases
    if not has_access and expected:
        has_read, reasons = syft_file._check_permission_with_reasons(user, "read")
        print(f"    Debug: {reasons[0] if reasons else 'No reasons'}")
    print()

## Chapter 3: Complex Multi-Level Scenarios

Real-world directory structures can be complex. Let's explore some challenging scenarios.

In [ ]:
# Create a complex project structure
cleanup_and_reset()

complex_structure = {
    "README.md": "Project readme",
    "src/main.py": "Main application",
    "src/models/user.py": "User model",
    "src/views/dashboard.py": "Dashboard view",
    "tests/test_main.py": "Main tests",
    "tests/unit/test_models.py": "Unit tests",
    "tests/integration/test_api.py": "Integration tests",
    "docs/user/guide.md": "User guide",
    "docs/dev/api.md": "Developer API docs",
    "docs/dev/internal/architecture.md": "Internal architecture",
    "config/dev/database.json": "Dev database config",
    "config/prod/database.json": "Prod database config",
    "scripts/deploy.sh": "Deployment script",
    "data/raw/input.csv": "Raw input data",
    "data/processed/clean.csv": "Processed data"
}

for rel_path, content in complex_structure.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

print("Created complex project structure with", len(complex_structure), "files")
print("\nDirectory tree:")
print("src/        (source code)")
print("tests/      (test files)")
print("docs/       (documentation)")
print("config/     (configuration)")
print("scripts/    (utility scripts)")
print("data/       (data files)")

In [ ]:
# Set up a realistic permission hierarchy

# Root: Basic project access
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "README.md"
  access:
    read:
    - "*"  # Everyone can read the README
- pattern: "**/*.py"
  access:
    read:
    - developers@example.com
""")

# Tests: More permissive for QA team
tests_yaml = tutorial_dir / "tests" / "syft.pub.yaml"
tests_yaml.write_text("""rules:
- pattern: "**/*.py"
  access:
    read:
    - qa_team@example.com
    write:
    - qa_team@example.com
""")

# Docs: Public documentation, restricted internal docs
docs_yaml = tutorial_dir / "docs" / "syft.pub.yaml"
docs_yaml.write_text("""rules:
- pattern: "user/**"
  access:
    read:
    - "*"  # Public user docs
- pattern: "dev/**"
  access:
    read:
    - developers@example.com
""")

# Config: Restricted by environment
config_yaml = tutorial_dir / "config" / "syft.pub.yaml"
config_yaml.write_text("""rules:
- pattern: "dev/**"
  access:
    read:
    - developers@example.com
    write:
    - developers@example.com
- pattern: "prod/**"
  access:
    read:
    - devops@example.com
    admin:
    - devops@example.com
""")

print("Set up realistic permission hierarchy:")
show_yaml(tutorial_dir, "Root: Basic access")
show_yaml(tutorial_dir / "tests", "Tests: QA team access")
show_yaml(tutorial_dir / "docs", "Docs: Public + dev access")
show_yaml(tutorial_dir / "config", "Config: Environment-based access")

In [ ]:
# Test the complex inheritance scenarios
print("\n=== Complex Inheritance Test Cases ===")

test_scenarios = [
    # (file, user, permission, expected, explanation)
    ("README.md", "anyone@example.com", "read", True, "Public README at root"),
    ("src/main.py", "developers@example.com", "read", True, "Python file, root permissions"),
    ("src/main.py", "qa_team@example.com", "read", False, "QA doesn't inherit from root for src/"),
    ("tests/test_main.py", "qa_team@example.com", "read", True, "QA has test access"),
    ("tests/test_main.py", "qa_team@example.com", "write", True, "QA can write tests"),
    ("tests/test_main.py", "developers@example.com", "read", False, "Devs blocked by nearest-node"),
    ("docs/user/guide.md", "anyone@example.com", "read", True, "Public user docs"),
    ("docs/dev/api.md", "developers@example.com", "read", True, "Dev docs for developers"),
    ("docs/dev/api.md", "anyone@example.com", "read", False, "Dev docs not public"),
    ("config/dev/database.json", "developers@example.com", "write", True, "Dev config writable"),
    ("config/prod/database.json", "devops@example.com", "admin", True, "DevOps controls prod"),
    ("config/prod/database.json", "developers@example.com", "read", False, "Devs can't read prod"),
]

for rel_path, user, permission, expected, explanation in test_scenarios:
    full_path = tutorial_dir / rel_path
    syft_file = sp.open(full_path)
    
    has_perm = getattr(syft_file, f"has_{permission}_access")(user)
    status = "✓" if has_perm == expected else "✗ WRONG"
    
    print(f"{status} {rel_path}")
    print(f"    {user} → {permission}: {has_perm} ({explanation})")
    
    if has_perm != expected:
        has_access, reasons = syft_file._check_permission_with_reasons(user, permission)
        print(f"    DEBUG: {reasons[0] if reasons else 'No reasons'}")
    print()

## Chapter 4: Understanding "No Accumulation"

One of the most important concepts: SyftBox does **NOT** accumulate permissions from multiple directories. The nearest node with matching rules wins completely.

In [ ]:
# Demonstrate the "no accumulation" principle
cleanup_and_reset()

# Create a simple 3-level structure
grandparent = tutorial_dir / "grandparent"
parent = grandparent / "parent" 
child = parent / "child"
target_file = child / "target.txt"

target_file.parent.mkdir(parents=True)
target_file.write_text("Target file for testing")

print("Created 3-level structure:")
print("grandparent/parent/child/target.txt")

In [ ]:
# Set permissions at each level with DIFFERENT users

# Grandparent: Alice has admin
grandparent_yaml = grandparent / "syft.pub.yaml"
grandparent_yaml.write_text("""rules:
- pattern: "**"
  access:
    admin:
    - alice@example.com
""")

# Parent: Bob has write (but not admin!)
parent_yaml = parent / "syft.pub.yaml"
parent_yaml.write_text("""rules:
- pattern: "**"
  access:
    write:
    - bob@example.com
""")

show_yaml(grandparent, "Grandparent: Alice admin")
show_yaml(parent, "Parent: Bob write")

print("\n=== No Accumulation Test ===")
print("If permissions accumulated, target.txt would have:")
print("- Alice: admin (from grandparent)")
print("- Bob: write (from parent)")
print("\nBut SyftBox uses nearest-node, so only parent rules apply:")

In [ ]:
# Test the actual behavior
syft_target = sp.open(target_file)

print("\nActual permissions on target.txt:")
print(f"Alice admin: {syft_target.has_admin_access('alice@example.com')}")
print(f"Alice write: {syft_target.has_write_access('alice@example.com')}")
print(f"Alice read:  {syft_target.has_read_access('alice@example.com')}")
print()
print(f"Bob admin:   {syft_target.has_admin_access('bob@example.com')}")
print(f"Bob write:   {syft_target.has_write_access('bob@example.com')}")
print(f"Bob read:    {syft_target.has_read_access('bob@example.com')}")

print("\n🔍 Analysis:")
print("✓ Bob has write access (from parent - nearest node)")
print("✗ Alice has NO access (grandparent ignored due to nearest-node)")
print("\nThe parent directory 'wins' completely, blocking grandparent permissions.")

In [ ]:
# Show the detailed reasoning
print("\n=== Detailed Permission Reasoning ===")

for user in ["alice@example.com", "bob@example.com"]:
    print(f"\n{user}:")
    has_admin, admin_reasons = syft_target._check_permission_with_reasons(user, "admin")
    print(f"  Admin: {has_admin}")
    for reason in admin_reasons:
        print(f"    • {reason}")
    
    has_write, write_reasons = syft_target._check_permission_with_reasons(user, "write")
    print(f"  Write: {has_write}")
    for reason in write_reasons:
        print(f"    • {reason}")

## Chapter 5: Debugging Inheritance Issues

When inheritance doesn't work as expected, here's how to debug it systematically.

In [ ]:
# Create a problematic scenario
cleanup_and_reset()

# User reports: "Alice should have access to reports/data.csv but doesn't!"
problem_file = tutorial_dir / "reports" / "data.csv"
problem_file.parent.mkdir(parents=True)
problem_file.write_text("Important data")

# User created this permission thinking it would work
root_yaml = tutorial_dir / "syft.pub.yaml"
root_yaml.write_text("""rules:
- pattern: "*.csv"
  access:
    read:
    - alice@example.com
""")

print("=== Debugging Inheritance Problem ===")
print(f"Problem: Alice can't access {problem_file.relative_to(tutorial_dir)}")
print(f"User expects: *.csv pattern should match")

show_yaml(tutorial_dir, "User's permission rule")

# Test the actual access
syft_problem = sp.open(problem_file)
has_access = syft_problem.has_read_access("alice@example.com")
print(f"\nActual result: Alice has access = {has_access}")
print("❌ Bug confirmed!")

In [ ]:
# Debug step 1: Check the reasoning
print("\n=== Debug Step 1: Check Permission Reasoning ===")

has_read, reasons = syft_problem._check_permission_with_reasons("alice@example.com", "read")
print(f"Has read access: {has_read}")
print("Reasons:")
for reason in reasons:
    print(f"  • {reason}")

print("\n💡 Diagnosis: 'No permission found' means pattern didn't match")

In [ ]:
# Debug step 2: Analyze the pattern mismatch
print("\n=== Debug Step 2: Pattern Analysis ===")
print(f"File path: reports/data.csv")
print(f"Pattern:   *.csv")
print()
print("Problem identified:")
print("❌ '*.csv' only matches files directly in the root directory")
print("❌ 'reports/data.csv' has a subdirectory, so pattern doesn't match")
print()
print("Solutions:")
print("1. Use '**/*.csv' to match .csv files at any depth")
print("2. Use 'reports/*.csv' to match .csv files specifically in reports/")
print("3. Move the file to root directory (not always practical)")

In [ ]:
# Apply the fix
print("\n=== Applying Fix ===")

# Fix: Use recursive pattern
root_yaml.write_text("""rules:
- pattern: "**/*.csv"
  access:
    read:
    - alice@example.com
""")

show_yaml(tutorial_dir, "Fixed permission rule")

# Test the fix
syft_fixed = sp.open(problem_file)
has_access = syft_fixed.has_read_access("alice@example.com")
print(f"\nAfter fix: Alice has access = {has_access} ✓")

# Verify the reasoning
has_read, reasons = syft_fixed._check_permission_with_reasons("alice@example.com", "read")
print(f"Reason: {reasons[0] if reasons else 'No reason'}")

## Chapter 6: Debugging Checklist

A systematic approach to debugging inheritance issues:

In [ ]:
def debug_inheritance_issue(file_path, user, permission="read"):
    """Systematic debugging function for inheritance issues"""
    print(f"\n=== Debugging Inheritance for {file_path.relative_to(tutorial_dir)} ===")
    print(f"User: {user}")
    print(f"Permission: {permission}")
    print()
    
    syft_file = sp.open(file_path)
    
    # Step 1: Check current access
    has_access = getattr(syft_file, f"has_{permission}_access")(user)
    print(f"1. Current access: {has_access}")
    
    # Step 2: Get detailed reasoning
    has_perm, reasons = syft_file._check_permission_with_reasons(user, permission)
    print(f"2. Detailed reasoning:")
    for reason in reasons:
        print(f"   • {reason}")
    
    # Step 3: Check for YAML files in directory tree
    print(f"3. YAML files in inheritance path:")
    current = file_path.parent
    yaml_files = []
    while current != tutorial_dir.parent:
        yaml_file = current / "syft.pub.yaml"
        if yaml_file.exists():
            yaml_files.append(current)
        current = current.parent
    
    if yaml_files:
        for yaml_dir in yaml_files:
            rel_path = yaml_dir.relative_to(tutorial_dir) if yaml_dir != tutorial_dir else "(root)"
            print(f"   • {rel_path}")
    else:
        print(f"   • No YAML files found in path")
    
    # Step 4: Suggestions
    print(f"4. Debugging suggestions:")
    if not has_access:
        if "No permission found" in str(reasons):
            print(f"   • No matching pattern found - check pattern syntax")
            print(f"   • File: {file_path.relative_to(tutorial_dir)}")
            print(f"   • Try patterns: **/{file_path.name}, {'/'.join(file_path.parts[-2:])}, **")
        elif "terminal" in str(reasons).lower():
            print(f"   • Blocked by terminal node - check terminal: true settings")
        else:
            print(f"   • Check user spelling and permission levels")
    else:
        print(f"   • Access working correctly")
    
    return has_access

# Test the debugging function
debug_inheritance_issue(problem_file, "alice@example.com")

## Summary: Part 3 Key Concepts

Outstanding! You've mastered SyftBox inheritance and hierarchy:

### ✅ **Core Algorithm Mastered**
1. **Nearest-Node Algorithm**: Find the closest parent directory with matching rules
2. **No Accumulation**: Nearest node completely replaces any parent permissions
3. **Pattern Matching Required**: Directory must have rules that match the file
4. **Single Rule Selection**: First matching rule within a directory wins

### ✅ **Inheritance Principles**
1. **Upward Search**: Start at file's directory, walk up the tree
2. **First Match Wins**: Stop at first directory with matching pattern
3. **Complete Override**: Nearest node blocks all parent permissions
4. **Pattern Specificity**: More specific patterns take precedence

### ✅ **Debugging Skills**
1. Use `_check_permission_with_reasons()` for detailed analysis
2. Check inheritance path for YAML files
3. Verify patterns match file paths correctly
4. Look for "No permission found" vs other denial reasons

### ✅ **Common Issues Solved**
1. **Pattern too specific**: Use `**` for recursive matching
2. **Wrong directory level**: Permissions inherit from nearest matching node
3. **Expecting accumulation**: Remember nearest-node overrides everything
4. **Case sensitivity**: Patterns must match exact case

### 🚀 **Next Up: Part 4 - Terminal Nodes and Blocking**
In the next tutorial, you'll learn:
- How to stop inheritance with terminal nodes
- Advanced inheritance control
- Creating permission boundaries
- Complex access control scenarios

You're becoming a SyftBox permissions expert!

In [ ]:
# Cleanup
shutil.rmtree(tutorial_dir)
print("Tutorial 3 complete! 🎉")
print("Ready for Part 4: Terminal Nodes and Blocking")